# Data Acquisition from CDLI


# 0 Import Packages

In [ ]:
import requests
import tqdm
import os
import errno
import pandas as pd
import csv

# 1. Create Download Directory
Create a directory called `cdlidata`. If the directory already exists, do nothing.

For the code, see [Stack Overflow](http://stackoverflow.com/questions/18973418/os-mkdirpath-returns-oserror-when-directory-does-not-exist).

In [ ]:
try:
    os.mkdir('cdlidata')
except OSError as exc:
    if exc.errno !=errno.EEXIST:
        raise
    pass

# 2 Download the Files

In [ ]:
files = ["cdliatf_unblocked.atf", "cdli_catalogue_1of2.csv", "cdli_catalogue_2of2.csv"]
CHUNK = 16 * 1024
for file in files:
    url = "https://github.com/cdli-gh/data/raw/master/" + file
    file = 'cdlidata/' + file
    with requests.get(url, stream=True) as r:
        if r.status_code == 200:
            print("Downloading " + url + " saving as " + file)
            with open(file, 'wb') as f:
                for c in tqdm.tqdm(r.iter_content(chunk_size=CHUNK)):
                    f.write(c)
        else:
            print(url + " does not exist.")

# Concatenate the Catalogue Files

In [ ]:
filenames = files[1:]
with open('cdlidata/catalogue.csv', 'w', encoding="utf8") as outfile:
    for fname in filenames:
        with open('cdlidata/' + fname, encoding="utf8") as infile:
            for line in infile:
                outfile.write(line)

At the time of writing this notebook, the file cdli_catalogue_1of2.csv had at least one instance of a a double record in a single line, resulting in more thatn the standard 63 fields. This makes it necessary to read the file with the `csv` library and simply delete all columns higher than 63.

In [ ]:
catalogue = []
with open('cdlidata/catalogue.csv', 'r', encoding="utf8") as f:
    csv_reader = csv.reader(f, delimiter=',', quotechar='"')
    count = 0
    for row in csv_reader:
        catalogue.append(row[:63])

In [ ]:
df = pd.DataFrame(catalogue)
col_names = df.iloc[0]
df = df[1:]
df = df.rename(columns = col_names)
df